In [ ]:
import requests
import json
import pandas as pd

# Read the API key from the text file - update file path to your api_key_file location
api_key_file = r"...\Code\rapidapi_key.txt"
with open(api_key_file, 'r') as file:
    rapidapi_key = file.read().strip()

# Define the URL and headers
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"
headers = {
    'x-rapidapi-key': rapidapi_key,
    'x-rapidapi-host': 'zillow-com1.p.rapidapi.com',
    'Content-Type': 'application/octet-stream'
}

# Initialize an empty list to store all property details
all_properties = []

# Loop through price ranges to  maximize data collection
for min_price in range(100000, 700000, 20000):
    max_price = min_price + 19999.99
    
    # Loop through pages 1 to 20 for each price range
    for page in range(1, 21):
        # Define the query parameters for the current price range and page
        queryString = {
            "location": "ct",
            "page": str(page),  # Convert page number to string
            "status_type": "RecentlySold",
            "home_type": "Houses,Condos,Townhomes",  # Ensure no spaces after commas
            "minPrice": str(min_price),
            "maxPrice": str(max_price),
            "soldInLast": "36m",
            "isForSaleForeclosure": "false"
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=queryString)
        
        # Check if the response status is OK (200)
        if response.status_code == 200:
            # Parse the response content
            content_response = response.json()
            
            # Debugging: Print the raw content response
            print(f"Raw content response for minPrice {min_price}, maxPrice {max_price}, page {page}:")
            print(json.dumps(content_response, indent=2))
            
            if 'props' in content_response and len(content_response['props']) > 0:
                # Append the properties to the all_properties list
                all_properties.extend(content_response['props'])
            else:
                print(f"No properties found for minPrice {min_price}, maxPrice {max_price}, page {page}.")
        else:
            print(f"Failed to get data for minPrice {min_price}, maxPrice {max_price}, page {page}. Status code: {response.status_code}")

# Convert the list of all properties to a DataFrame
properties_df = pd.DataFrame(all_properties)

# Define the path to save the CSV
csv_path = r"...App/Data/zillow_properties.csv"

# Save the DataFrame to a CSV file
properties_df.to_csv(csv_path, index=False)

print(f"Data saved to {csv_path}")